# Fine-Tuned

This notebook extracts respiration signals for each subject using the fine-tuned models.

In [ ]:
import respiration.utils as utils

# Fine tuned model ids
tuned_models = {
    '20240509_122327',
    '20240509_124125',
    '20240509_132544',
    '20240509_135603',
    '20240509_144815',
    '20240509_151734',
}

# Map model names to their paths
models = {
    'efficient_phys': utils.file_path('data', 'rPPG-Toolbox', 'BP4D_PseudoLabel_EfficientPhys.pth'),
}

manifests = []

# Add the best fine-tuned model weights to the model paths
for model_id in tuned_models:
    model_dir = utils.dir_path('models', 'fine_tuned', model_id)

    manifest_path = utils.dir_path(model_dir, 'manifest.json')
    manifest = utils.read_json(manifest_path)
    best_model = manifest['tuned_models'][-1]

    model_path = utils.join_paths(model_dir, best_model['model'])
    models[model_id] = model_path
    manifests.append(manifest)

utils.pretty_print(models)

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])

device = utils.get_torch_device()
dim = 72
frame_depth = 20

In [ ]:
import torch
import pandas as pd
import datetime as dt

from tqdm import tqdm
from respiration.extractor.efficient_phys import EfficientPhys

predictions = []

for (subject, setting) in tqdm(scenarios):
    print(f"Processing {subject} - {setting}")

    video_path = dataset.get_video_path(subject, setting)
    frame_count = utils.get_frame_count(video_path)
    chunk_size = (frame_count // frame_depth) * frame_depth - (frame_depth - 1)
    # chunk_size = frame_depth * 20 + 1

    frames, meta = utils.read_video_rgb(video_path, chunk_size)
    frames = utils.down_sample_video(frames, dim)
    frames = torch.tensor(frames, dtype=torch.float32, device=device).permute(0, 3, 1, 2)

    for (model_id, model_path) in models.items():
        print(f"--> Using {model_id} model")
        # Wrap modul in nn.DataParallel to fix the model loading issue
        model = torch.nn.DataParallel(EfficientPhys(img_size=dim, frame_depth=frame_depth))
        model.to(device)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()

        start = dt.datetime.now()

        with torch.no_grad():
            prediction = model(frames).cpu().detach().numpy().squeeze()

        predictions.append({
            'model': model_id,
            'subject': subject,
            'setting': setting,
            'duration': dt.datetime.now() - start,
            'chunk_size': chunk_size,
            'sampling_rate': meta.fps,
            'signal': prediction.tolist(),
        })

    del frames

predictions = pd.DataFrame(predictions)

# Store the predictions to csv
signals_dir = utils.dir_path('outputs', 'signals', mkdir=True)
signals_path = utils.join_paths(signals_dir, 'fine_tuned_predictions.csv')
predictions.to_csv(signals_path, index=False)

# Store the manifests
manifests_path = utils.join_paths(signals_dir, 'fine_tuned_manifest.json')

In [ ]:
predictions.head()